### MATH2504 S2, 2022, BigHW Submission 
### Submission form II of IV 

Student #1 name: Lief Lundmark-Aitcheson

Student #1 ID: s4703653

Student #2 name: Lucy Carra Schulz 

Student #2 ID: 47020674

GitHub repo: Lief-Lundmark-Aitcheson__Lucy-Carra-Schulz-2504-2022-BigHW

# Solution to [Question 5](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q5)

In [3]:
function get_factors(n)
    factors = [1]
    for i in 2:floor(sqrt(n))
        if n % i == 0
            push!(factors, i, n / i)
        end
    end
    return factors
end

perfects = [i for i in 1:10^6 if sum(get_factors(i))==i]

print("The first $(length(perfects)) perfect numbers are $(perfects)")

The first 5 perfect numbers are [1, 6, 28, 496, 8128]

# Solution to [Question 6](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q6)

In [5]:
using Plots
using StatsBase
using Profile
using ProfileView

function sieve_of_Eratosthenes(n)
    primebits = ones(Bool,n) #Will contain true if the index is prime (initially all assumed prime)
    primebits[1] = false #The number 1 is not prime
    p = 2 #Smallest prime
    while p ≤ n
        i = 2p
        while i ≤ n  # \le +[TAB]
            primebits[i] = false
            i += p
        end
        p += 1
        while p ≤ n && !primebits[p]
            p += 1
        end
    end
    (1:n)[primebits]
end

function run(n::Int64)
    even_range = 4:2:n
    primes ::Vector{Int64} = sieve_of_Eratosthenes(n)[2:end]
    #exclude 2, because odd sums are problematic for us
    pairs = zeros(Int64,2*length(even_range))
    pairs[1] = 1 #account for 2+2 = 4
    for (i, p) in enumerate(primes)
        for j in i:length(primes)
            sum = p + primes[j]
            pairs[sum÷2 - 1] += 1
        end
    end
    checks = pairs[1:length(even_range)]
    if 0 ∈ checks
        println("Found a counter example for Goldbach")
    end
    
    x = sample(even_range,min(2*10^4,n÷3),replace = false)
    y = checks[(-1 .+ x.÷2)]
    scatter(x,y,legend=false,xlabel="N",ylabel="Number of Goldbach pairs")
    png("scatter.png")
    return nothing
end

@time run(10^6)

  5.660514 seconds (786.03 k allocations: 33.744 MiB)


![](scatter.png)

# Solution to [Question 7](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q7)

In [ ]:
using Random

random_license_plate() = String(rand('0':'9',3)) * " " * String(rand('A':'Z',3))

function prepare_data(;N = 10^7)
    Random.seed!(1)
    indexes = rand(1:10^12, N)
    plates = [random_license_plate() for _ in 1:N]

    #One specific spot is set to a special value.
    special_spot = Int(floor(0.9*N)) #Arbitrary index
    indexes[special_spot], plates[special_spot] = 1234567890, "000 ZZZ"
    
    return indexes, plates
end

function prepare_sorted_data(;N = 10^7)
    indexes, plates = prepare_data(N=N)  
    perm = sortperm(indexes)
    return indexes[perm], plates[perm]
end

indexes, plates = prepare_sorted_data(N=5) #as an example set N=5
@show indexes
@show plates;

using BenchmarkTools

function find_special_plate(indexes, plates; special_index = 1234567890)
    for (i, index) in enumerate(indexes) 
        if index == special_index 
            return plates[i]
        end
    end 
    throw(Error("Index not found"))
end

indexes, plates = prepare_sorted_data()

@btime find_special_plate(indexes, plates)

## 7a:

The function random_license_plate() makes use of Julia's ability to define a function in one line. This is useful for simple functions as it can shorten the syntax as it avoids writing function, end and return. 

In [1]:
f(x)=x^2
f(3)

9


The strings were concatenated using the * operator which takes two strings and concatenates them into one. In Julia * was used for concatenation rather than + as string concatenation is a noncommutative operation.


In [2]:
first = "Lucy"
second = "Carra Schulz"
full = first*" "*second

"Lucy Carra Schulz"


The rand() function was used to generate the random digits and letters. The function takes an indexable collection, in this case '0':'9' and 'A':'Z', and a size (either a single argument for values or a tuple for dimensions) to randomize, in the case 3. The complexity of rand() is linear.



In [5]:
x = String(rand('0':'9',3))

"226"

The line “plates = [random_license_plate() for _ in 1:N]” makes an array of N random license plates by calling random_license_plate() N times. The code makes use of the bare underscore which is used when a variable must be defined but is never used.
    


In [7]:
for _ in 1:5
    println(1)
end

1
1
1
1
1


The function prepare_sorted_data returns a tuple separated by a comma which allows one function to return two values.
    


In [15]:
function coordinate()
    return rand(0:9,1)[1], rand(0:9,1)[1]
end

x, y = coordinate()
@show x
@show y

x = 3


7

The function sortperm() returns a vector of indexes that would sort the given vector. This allows the plates and indexes to be sorted according to the order of indexes.

## 7b:

This code assigns $10^7$ license plates a random index between $0$ and $10^{12}$. The probability of one of the number $10^7$ number plates being assigned index $1234567890$ is $1-(\frac{10^{12}-1}{10^{12}})^{10^7} \approx 1^{-6}$. 

The @btime macro from the BenchmarkTools package prints the minimum time and memory allocation before returning the output of the function. For the find_special_plate function the output is 3.671 μs (0 allocations: 0 bytes) "000 ZZZ". 

The package has 306 commits. The most commented on issue from the repo is "stop printing "humanized" time units #125". This issue points out that benchmark puts the output of its macros in ms, μs or ns depending on the output. This makes comparing times hard and confusing. 

    On Slack today, some timings were posted:

    julia> @btime sum(a) > 0
      26.323 μs (1 allocation: 16 bytes)
    false

    julia> @btime any(a .> 0)
      116.147 μs (5 allocations: 16.64 KiB)
    true

    julia> @btime any(!iszero, a)
      178.574 ns (0 allocations: 0 bytes)
    true
    The entire ensuing conversation was predicated on the fact that any(!iszero, a) was mysteriously slower than both sum(a) > 0 and any(a .> 0). Nobody noticed that the last time measurement is in nanoseconds while the first two are in microseconds. I've seen this happen quite often. It's happened to me. The reason I knew to be alert to this is because I've missed the very subtle difference between "ms", "μs" and "ns" before. Moreover, this doesn't match how Base prints times anymore:

    julia> @time sleep(1)
      1.047126 seconds (114.42 k allocations: 6.038 MiB, 0.90% gc time)

    julia> @time sleep(0.001)
      0.006039 seconds (82 allocations: 5.375 KiB)
    In fact, Base's time printing was changed for the exact same reason! I propose that we choose one of the following ways to print times instead:

      0.000026323 seconds (1 allocation: 16 bytes)
      0.000116147 seconds (5 allocations: 16.64 KiB)
      0.000000179 seconds (0 allocations: 0 bytes)
    In other words, with a fixed 9 decimal digits and room for up to three leading digits. That's plenty of digits and it's very clear what scale values are on. If the values must be scaled somehow, then consider using engineering notation instead, so something like this:

      26.323e-6 seconds (1 allocation: 16 bytes)
    116.147e-6 seconds (5 allocations: 16.64 KiB)
    179.000e-9 seconds (0 allocations: 0 bytes)
    But note that this is not much shorter and far less clear than just printing 9 digits after the decimal.

The @benchmark macro takes 10000 samples and returns the range, mean and median of the time as well as a histogram, as opposed to the @btime macro which just returns the minimum time.

## 7c:

In [2]:
# BROKEN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

using Random

random_license_plate() = String(rand('0':'9',3)) * " " * String(rand('A':'Z',3))

function prepare_data(;N = 10^7)
    Random.seed!(1)
    indexes = rand(1:10^12, N)
    plates = [random_license_plate() for _ in 1:N]

    #One specific spot is set to a special value.
    special_spot = Int(floor(0.9*N)) #Arbitrary index
    indexes[special_spot], plates[special_spot] = 1234567890, "000 ZZZ"
    
    return indexes, plates
end

function prepare_sorted_data(;N = 10^7)
    indexes, plates = prepare_data(N=N)  
    perm = sortperm(indexes)
    return indexes[perm], plates[perm]
end



using BenchmarkTools

function find_special_plate(indexes, plates; special_index = 1234567890)
    L = 0
    R = length(plates)-1
    while L <= R
        m =(L+R)÷2
        if indexes[m] < special_index
            L=m+1
        end
        if indexes[m] > special_index
            R=m-1
        end
        if indexes[m] == special_index
            return m
        end
    end 
    throw(Error("Index not found"))
end

indexes, plates = prepare_sorted_data()

@benchmark find_special_plate(indexes, plates)


BenchmarkTools.Trial: 10000 samples with 930 evaluations.
 Range (min … max):  108.871 ns …   8.124 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     114.919 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   120.813 ns ± 117.423 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃    ▅█                                                       
  ▂█▄▆▆▆███▆▆██▆▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▃
  109 ns           Histogram: frequency by time          154 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

## 7d:

Searching via binary search in a sorted array is normally $O(log(n))$ time complexity because the searching successively halves the size of the problem whereas the original code was $O(n)$ time comlexity.  

# Solution to [Question 8](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q8)

## 8a:

The code below has been modified to sort by length of last name:

In [2]:
function array_table(array, heading)
    println(heading,":")
    for (i,a) in enumerate(array)
        println(i,"\t",a)
    end
end

names_list = [   "Amy Chan",
            "Maithili Mehta",
            "Anna Foeglein",
            "Andy Ferris",
            "Thomas Graham",
            "Elaine Schenk",
            "Jesse Woods",
            "Tina Moghaddam",
            "Paul Bellette",
            "Paul Vrbik",
            "Tom Cranitch",
            "Yoni Nazarathy",
            "Sam Hambleton",
            "Alistair Falconer",
            "Emma Comino",
            "Ivana Carrizo-Molina"]

sorted_names = sort(names_list)
array_table(sorted_names, "Sorted by first name")

println()

sorted_names = sort(names_list,by=(x)->length(split(x," ")[2]))
array_table(sorted_names,"Sorted by length of last name")

Sorted by first name:
1	Alistair Falconer
2	Amy Chan
3	Andy Ferris
4	Anna Foeglein
5	Elaine Schenk
6	Emma Comino
7	Ivana Carrizo-Molina
8	Jesse Woods
9	Maithili Mehta
10	Paul Bellette
11	Paul Vrbik
12	Sam Hambleton
13	Thomas Graham
14	Tina Moghaddam
15	Tom Cranitch
16	Yoni Nazarathy



Sorted by length of last name:
1	Amy Chan
2	Maithili Mehta
3	Jesse Woods
4	Paul Vrbik
5	Andy Ferris
6	Thomas Graham
7	Elaine Schenk
8	Emma Comino
9	Anna Foeglein
10	Paul Bellette
11	Tom Cranitch
12	Alistair Falconer
13	Tina Moghaddam
14	Yoni Nazarathy
15	Sam Hambleton
16	Ivana Carrizo-Molina


## 8b:

In [14]:
function sort_bubble(list; by=(x)->x)
    for i in 1:length(list)-1
        for j in 1:length(list)-i
            if by(list[j]) > by(list[j+1])
                tmp = list[j]
                list[j] = list[j+1]
                list[j+1] = tmp
            end
        end
    end
    return list
end

sorted_names = sort_bubble(names_list,by=(x)->reverse(split(x," ")))
array_table(sorted_names,"Bubble sorted by last name")

Bubble sorted by last name:
1	Paul Bellette
2	Ivana Carrizo-Molina
3	Amy Chan
4	Emma Comino
5	Tom Cranitch
6	Alistair Falconer
7	Andy Ferris
8	Anna Foeglein
9	Thomas Graham
10	Sam Hambleton
11	Maithili Mehta
12	Tina Moghaddam
13	Yoni Nazarathy
14	Elaine Schenk
15	Paul Vrbik
16	Jesse Woods


## 8c:

In [4]:
#first implementation of quicksort, not in place
function quick_sort(list;by=(x)->x)
    if length(list)<=1
        return list
    end
    pivot = by(list[1]) #The choice of pivot can be optimised but this is sufficient for our purposes
    left = [x for x in list[2:end] if by(x)<pivot]
    right = [x for x in list[2:end] if by(x)>=pivot]
    return(cat(quick_sort(left,by=by),[list[1]],quick_sort(right,by=by),dims=1))
end 

#in place implementation, far more memory efficient
function quick_sort_in_place(list,lo=1,hi=length(list);by=(x)->x)
    if lo>=hi || lo < 1
        return list
    end
    pivot = by(list[lo])
    i = lo+1
    for j in lo+1:hi
        if by(list[j])<pivot
            list[i],list[j] = list[j],list[i]
            i += 1
        end
    end
    list[lo],list[i-1] = list[i-1],list[lo]
    quick_sort_in_place(list,lo,i-2,by=by)
    quick_sort_in_place(list,i,hi,by=by)
    return list
end

sorted_names = quick_sort_in_place(names_list,by=(x)->reverse(split(x," ")))
array_table(sorted_names,"Quick sorted by last name")

Quick sorted by last name:
1	Paul Bellette
2	Ivana Carrizo-Molina
3	Amy Chan
4	Emma Comino
5	Tom Cranitch
6	Alistair Falconer
7	Andy Ferris
8	Anna Foeglein
9	Thomas Graham
10	Sam Hambleton
11	Maithili Mehta
12	Tina Moghaddam
13	Yoni Nazarathy
14	Elaine Schenk
15	Paul Vrbik
16	Jesse Woods


## 8d:

In [6]:
using BenchmarkTools
function test(n)
    data = rand(Int,n)
    println("system sort")
    @btime sort($data)
    println("quick sort")
    @btime quick_sort_in_place($data)
    return nothing
end

test(10^4) #larger values take a long time to run because btime takes multiple samples


system sort


  549.958 μs (2 allocations: 78.17 KiB)
quick sort


  38.908 ms (0 allocations: 0 bytes)


Bubble sort has $O(n^2)$ time complexity, which means that the running time tends to grow quadratically with the size of the input. We can test with some smaller values and fit a quadratic:

![](bubble_sort.png)

Extrapolating this trend gives an estimated running time of 8.5 minutes for $10^6$ items, and 1400 hours for $10^8$ items, which is unreasonable to run.

We can however run trials for quick sort and the system sort. I obtained the following results:

$10^6$ items:

 - System sort = 89.285 ms
 - Quick sort = 132.084 ms

$10^7$ items:

 - System sort = 1.050 s
 - Quick sort = 2.107 s

$10^8$ items:

 - System sort = 12.082 s
 - Quick sort = 33.362 s

# Solution to [Question 9](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q9)

In [ ]:
using LinearAlgebra
using Random

function my_mult_inner_products(A,B)
    nA,mA = size(A)
    nB,mB = size(B)
    mA == nB
    n, m, p = nA, mA, mB
    C = Array{Float64}(undef,n,p)

    for i in 1:n
        for j in 1:p
            C[i,j] = A[i,:]' * B[:,j] #compute inner product of i'th row of A and j'th column of B
        end
    end
    return C
end

Random.seed!(0)
A = round.(10*rand(2,3))
B = round.(10*rand(3,4))
my_mult_inner_products(A,B)

## 9a:

In [17]:
using LinearAlgebra
using Random

function my_mult_by_cols(A,B)
    nA,mA = size(A)
    nB,mB = size(B)
    @assert mA == nB
    n, m, p = nA, mA, mB
    C = Array{Float64}(undef,n,p)

    for j in 1:p #4 columns
        # C[:,j]=Array{Float64}(undef,n,1)
        for q in 1:m
            C[:,j] += A[:,q]*B[q,j]
        end
    end
    return C
end

Random.seed!(0)
A = round.(10*rand(2,3))
B = round.(10*rand(3,4))
my_mult_by_cols(A,B)

## 9b:

In [18]:
using LinearAlgebra
using Random

function my_mult_by_rows(A,B)
    nA,mA = size(A)
    nB,mB = size(B)
    @assert mA == nB
    n, m, p = nA, mA, mB
    C = Array{Float64}(undef,n,p)

    for i in 1:n #2 rows
        C[i,:]=Array{Float64}(undef,1,p)
        for q in 1:m
            C[i,:] +=B[q,:]*A[i,q]
        end
    end
    return C
end

Random.seed!(0)
A = round.(10*rand(2,3))
B = round.(10*rand(3,4))
my_mult_by_rows(A,B)

## 9c:

In [19]:
using LinearAlgebra
using Random

function my_mult_sum_outer_products(A,B)
    nA,mA = size(A)
    nB,mB = size(B)
    @assert mA == nB
    n, m, p = nA, mA, mB
    C = Array{Float64}(undef,n,p)

    for q in 1:m #2 rows
        C += A[:,q]*B[q,:]'
    end
    return C
end

Random.seed!(0)
A = round.(10*rand(2,3))
B = round.(10*rand(3,4))
my_mult_sum_outer_products(A, B)

## 9d:


Both the original code, 9a and 9b use a nested loop. 9c only uses a single loop however it still has the same time complexity because the matrix multiplication has a hidden loop.